In [73]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import *
import matplotlib.pyplot as plt
import numpy as np
from qiskit.circuit.library import QFT

def qft(n):
    """
    Constructs the Quantum Fourier Transform (QFT) circuit for n qubits.

    Args:
        n (int): The number of qubits for the QFT circuit.

    Returns:
        qiskit.QuantumCircuit: The QFT circuit.
    """
    if n == 0:
        return QuantumCircuit(0)

    qc = QuantumCircuit(n)

    # Apply the QFT rotations
    for j in range(n):
        qc.h(j)  #apply the Hardamard to evry qubit at start
        for k in range(j + 1, n): # next unitl last qubit apply CP(theta,control-qubit,target-qubit)- control phase shift gate of 
            qc.cp(np.pi/2**(k-j), k, j)

    # Swap the qubits to reverse the order
    for i in range(n//2):
        qc.swap(i, n-i-1)

    qc.name = f"QFT({n})"
    return qc


In [75]:
from qiskit.quantum_info import * 

"""
Apply the QFT circuit on the basis state |5⟩for n= 4. Print the resulting quantum statevector and
verify that the amplitudes match the expected Fourier coefficients.
"""
# 1. Set up the parameters

n = 4
N = 2**n
input_state_decimal = 5

# 2. Create the quantum circuit
# The input state |5> corresponds to the binary string |0101>
# Qiskit uses a little-endian convention, so |q3 q2 q1 q0>
# To get |0101>, we apply X gates to q0 and q2.
qc = QuantumCircuit(n)
qc.x(0)
qc.x(2)
qc.barrier() # Just for visual separation

initial_state = Statevector.from_instruction(qc)

# 3. Apply the QFT

### Using QFT function from library
qft_circuit = QFT(num_qubits=n, do_swaps=False).decompose()

### using own implementation
#qft_circuit = qft(n)

qc.append(qft_circuit, range(n))
print("--- Circuit to transform |5> ---")
print(qc)


# qc.save_statevector() #explicity saving to retrive later as the state will collapse and lose this information after simulation

# 4. Simulate the circuit to get the final statevector
simulator = Aer.get_backend('statevector_simulator')
compiled_circuit = transpile(qc, simulator)
result = simulator.run(compiled_circuit).result()
simulated_statevector = result.get_statevector()

print("\n--- Simulation Results ---")
print("Resulting Statevector from Qiskit Simulation:")
# Print the statevector in a more readable format
for i, amp in enumerate(simulated_statevector):
    # a+bj is printed as (a, b)
    print(f"|{i:04b}> ({i}) : {amp.real:+.4f} + {amp.imag:+.4f}j")

# 5. Verify the amplitudes with the expected Fourier coefficients
print("\n--- Theoretical Verification ---")
print("Comparing simulated amplitudes with expected Fourier coefficients:")

all_match = True
for k in range(N):
    # Expected amplitude for the |k> state
    expected_amplitude = (1/np.sqrt(N)) * np.exp(2j * np.pi * input_state_decimal * k / N)
    simulated_amplitude = simulated_statevector[k]

    print(f"State |{k:04b}> ({k}):")
    print(f"  - Simulated: {simulated_amplitude.real:+.4f} + {simulated_amplitude.imag:+.4f}j")
    print(f"  - Expected:  {expected_amplitude.real:+.4f} + {expected_amplitude.imag:+.4f}j")

    # Check if the simulated and expected amplitudes are close
    if not np.allclose(simulated_amplitude, expected_amplitude):
        all_match = False
        print("  - MISMATCH FOUND!")

print("\n--- Verification Complete ---")
if all_match:
    print("✅ The simulated amplitudes successfully match the expected Fourier coefficients.")
else:
    print("❌ A mismatch was found between the simulated and expected results.")

--- Circuit to transform |5> ---
     ┌───┐ ░ ┌──────┐
q_0: ┤ X ├─░─┤0     ├
     └───┘ ░ │      │
q_1: ──────░─┤1     ├
     ┌───┐ ░ │  QFT │
q_2: ┤ X ├─░─┤2     ├
     └───┘ ░ │      │
q_3: ──────░─┤3     ├
           ░ └──────┘

--- Simulation Results ---
Resulting Statevector from Qiskit Simulation:
|0000> (0) : +0.2500 + -0.0000j
|0001> (1) : -0.2500 + +0.0000j
|0010> (2) : +0.0000 + +0.2500j
|0011> (3) : -0.0000 + -0.2500j
|0100> (4) : -0.1768 + -0.1768j
|0101> (5) : +0.1768 + +0.1768j
|0110> (6) : +0.1768 + -0.1768j
|0111> (7) : -0.1768 + +0.1768j
|1000> (8) : -0.0957 + +0.2310j
|1001> (9) : +0.0957 + -0.2310j
|1010> (10) : -0.2310 + -0.0957j
|1011> (11) : +0.2310 + +0.0957j
|1100> (12) : +0.2310 + -0.0957j
|1101> (13) : -0.2310 + +0.0957j
|1110> (14) : +0.0957 + +0.2310j
|1111> (15) : -0.0957 + -0.2310j

--- Theoretical Verification ---
Comparing simulated amplitudes with expected Fourier coefficients:
State |0000> (0):
  - Simulated: +0.2500 + -0.0000j
  - Expected:  +0.2500 +

/var/folders/1l/g3s1ytv57hv98hl39w041lxm0000gp/T/ipykernel_93594/3848946162.py:48: DeprecationWarning: The return type of saved statevectors has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.Statevector` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  for i, amp in enumerate(simulated_statevector):
